In [89]:
import pandas
import numpy
from numpy import std, correlate, sqrt
from scipy.stats import pearsonr
import statsmodels.api as sm
%run Table_Summary.ipynb

### Preparing data

In [2]:
filepath = 'D:/DEA/Data/Original_data.txt'
data = pandas.read_csv(filepath,sep='\t')
data.rename(columns={'origi':'Id'},inplace=True)
print(data.shape,'from 2019_05_25_Results.xlxs')
print(data.Year.unique(),'Years')
print(data.Id.nunique(),'Number of firms')
filepath = 'D:/DEA/Data/Database_secondreviewJIBS.dta' #
data1 = pandas.read_stata(filepath)
data1.rename(columns={'id':'Id','year':'Year','country':'Country'},inplace=True)
data1 = data1[['Id','Year','Country']]
data1['Year'] = pandas.to_numeric(data1.Year.astype(str).str[0:4])
print(data1.shape,'to get country')
data = pandas.merge(left=data,right=data1,how='left',on=['Year','Id'])
print(data.Country.unique())
filepath = 'D:/DEA/Data/2019_05_25_Results_updated.txt'
data2 = pandas.read_csv(filepath,sep='\t')
print(data2.shape,'correction by Elio of market commonality')
data2.rename(columns={'origi':'Id'},inplace=True)
data2 = data2[['Id','Year','Market_commonality']]
data.drop('Market_commonality',axis=1,inplace = True)
data = pandas.merge(left=data,right=data2,how='inner',on=['Year','Id'])
print(data.Non_RD_alliances.sum(),'Total number of Non-RD alliances')
print(data.RD_alliances.sum(),'Total number of RD alliances')

(1232, 33) from 2019_05_25_Results.xlxs
[1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004
 2005 2006 2007 2008 2009 2010 2011 2012] Years
56 Number of firms
(1539, 3) to get country
[nan 'USA' 'Switzerland' 'Netherlands' 'Germany' 'South Africa' 'Japan'
 'United Kingdom' 'France' 'Australia' 'Ireland' 'Spain' 'Denmark'
 'Sweden']
(1232, 34) correction by Elio of market commonality
4089.0 Total number of Non-RD alliances
7676.0 Total number of RD alliances


### Creating variables

In [122]:
df = data.copy()
df.reset_index(inplace=True,drop=True)
df = df.sort_values(by=['Id','Year'], axis=0, ascending=True)
# ASPIRATIONS
df['SA']=df.groupby(['Year']).ROE.transform('mean')
df['distance'] = df['ROE'] - df['SA']
df.loc[df.distance<0,'BSA']=df['distance']
df.loc[(df.distance>0)|(df.distance==0),'BSA']=0
df.loc[(df.distance>0)|(df.distance==0),'ASA']=df['distance']
df.loc[df.distance<0,'ASA']=0
# DURATION
df['DURATION'] = 0
mylist = []
for i in range(df.Id.nunique()):
  mylist.append(df.loc[df.Id==i+1,['Id','Year','BSA','DURATION']].values.tolist())
m = [0]
for firm in range(df.Id.nunique()):
  for i in mylist[firm]: 
    if i[2]<0:
      m.append([i[0],i[1],m[-1][2]+1])
    else:
      m.append([i[0],i[1],i[3]])
del m[0]
print(len(m))
mydf = pandas.DataFrame(m,columns=['Id','Year','DURATION'])
df = df.drop('DURATION',axis=1)
df = pandas.merge(left=df,right=mydf,how='inner',on=['Year','Id'])
print(df.loc[(df.Id==38)|(df.Id==39),['Id','Year','BSA','DURATION']])
df['BSA0'] = df.groupby(['Id']).BSA.shift(1)
df['ASA0'] = df.groupby(['Id']).ASA.shift(1)
df['DURATION0'] = df.groupby(['Id']).DURATION.shift(1)
# ATTENTION
filepath = 'D:/DEA/Data/Results_def_01_for_Elio.txt'
data5 = pandas.read_csv(filepath,sep='\t')
data5 = data5.iloc[1:1008,[0,1,25,26,27]]
data5.columns = ['Year','Id','PRODUCT','GEOGRAPHY','GROWTH']
data5['Year'] = pandas.to_numeric(data5.Year.astype(str).str[0:4])
df = pandas.merge(left=df,right=data5,how='inner',on=['Year','Id'])
df['ATTENTION'] = df[['PRODUCT','GEOGRAPHY','GROWTH']].std(axis=1)
# AMBIDEXTERITY
print(df.Non_RD_alliances.corr(df.RD_alliances),'correlation between RD alliances and non-RD alliances')
df['AMB0'] = df.RD_alliances + df.Non_RD_alliances
df['AMB1'] = df.RD_alliances - df.Non_RD_alliances
df['AMB1'] = df['AMB1'].abs()
# SIZE
df['Size'] = numpy.log(df.Employees) 
df[0:20]

1232
     Id  Year       BSA  DURATION
814  38  1991  0.000000       0.0
815  38  1992  0.000000       0.0
816  38  1993  0.000000       0.0
817  38  1994  0.000000       0.0
818  38  1995 -0.014391       1.0
819  38  1996 -0.098000       2.0
820  38  1997  0.000000       0.0
821  38  1998 -0.209075       1.0
822  38  1999 -0.059585       2.0
823  38  2000 -0.124833       3.0
824  38  2001 -0.008870       4.0
825  38  2002  0.000000       0.0
826  38  2003  0.000000       0.0
827  38  2004 -0.045434       1.0
828  38  2005 -0.000642       2.0
829  38  2006 -0.000154       3.0
830  38  2007 -0.634098       4.0
831  38  2008 -0.259360       5.0
832  38  2009 -0.163826       6.0
833  38  2010 -0.089444       7.0
834  38  2011 -0.104333       8.0
835  38  2012 -0.025738       9.0
836  39  1991       NaN       0.0
837  39  1992       NaN       0.0
838  39  1993       NaN       0.0
839  39  1994       NaN       0.0
840  39  1995       NaN       0.0
841  39  1996 -0.111000       1.0
842  39  

,Id,Year,Age,Employees,Total_assets,Net_sales,Human_resources_1,Human_resources_2,Physical_resources_1,Physical_resources_2,...,BSA0,ASA0,DURATION0,PRODUCT,GEOGRAPHY,GROWTH,ATTENTION,AMB0,AMB1,Size
0,1,1996,94.0,74289.0,13364.000,14236.000,0.192,5.218,4844.000,0.340,...,-0.038391,0.000000,1.0,0.0,0.784127,0.215873,0.405045,1.0,1.0,11.215718
1,1,1997,95.0,75639.0,13238.000,15070.000,0.199,5.019,5034.000,0.334,...,0.000000,0.049000,0.0,0.0,0.774649,0.225351,0.398454,0.0,0.0,11.233727
2,1,1998,96.0,73564.0,14153.000,15021.000,0.204,4.897,5566.000,0.371,...,0.000000,0.228423,0.0,0.0,0.750883,0.249117,0.382460,3.0,1.0,11.205911
3,1,1999,97.0,70549.0,13896.000,15659.000,0.222,4.505,5656.000,0.361,...,-0.120075,0.000000,1.0,0.0,0.793790,0.206210,0.411881,0.0,0.0,11.164063
4,1,2000,98.0,75000.0,14522.000,16724.000,0.223,4.485,5823.000,0.348,...,0.000000,0.092415,0.0,0.0,0.788899,0.211101,0.408407,0.0,0.0,11.225243
5,1,2001,99.0,71669.0,14606.000,16079.000,0.224,4.457,5615.000,0.349,...,0.000000,0.115167,0.0,0.0,0.817698,0.182302,0.429262,3.0,3.0,11.179814
6,1,2002,100.0,68774.0,15329.000,16332.000,0.237,4.211,5621.000,0.344,...,0.000000,0.040130,0.0,0.0,0.807039,0.192961,0.421434,3.0,1.0,11.138581
7,1,2003,101.0,67072.0,17600.000,18232.000,0.272,3.679,5609.000,0.308,...,0.000000,0.188444,0.0,0.0,0.788795,0.211205,0.408333,1.0,1.0,11.113522
8,1,2004,102.0,67071.0,20708.000,20011.000,0.298,3.352,5711.000,0.285,...,0.000000,0.152906,0.0,0.0,0.786930,0.213070,0.407016,2.0,2.0,11.113507
9,1,2005,103.0,69315.0,20513.000,21167.000,0.305,3.275,5593.000,0.264,...,0.000000,0.132566,0.0,0.0,0.841605,0.158395,0.447244,1.0,1.0,11.146417


### Correlation

In [103]:
mydata = df[['AMB0','AMB1','BSA','DURATION','ATTENTION','ASA','Absorbed_slack','Unabsorbed_slack','Absorptive_capacity','Market_commonality','Size','Year']]
print(len(mydata),'Number of observations')
ds_mean = mydata.agg(['mean','std']).T
corr = mydata.corr()
corr = pandas.concat([ds_mean,corr], axis = 1)
corr = corr.applymap('{:.2f}'.format)
corr['statistics'] = ' '
corr.set_index('statistics',append=True,inplace=True)
print(corr)

def pearsonr_pval(x,y):
  return pearsonr(x,y)[1].round(3)
pval = mydata.corr(method=pearsonr_pval)
pval = pval.applymap('{:.3f}'.format)
pval = pval.applymap(lambda x: '(' + x + ')')
pval['statistics'] = 'p-value'
pval.set_index('statistics',append=True,inplace=True)

corr = pandas.concat([corr,pval],axis=0).sort_index(kind='merge').reindex(['AMB0','AMB1','BSA','DURATION','ATTENTION','ASA','Absorbed_slack','Unabsorbed_slack','Absorptive_capacity','Market_commonality','Size','Year'],axis=0,level=0)
filepath = 'D:/DEA/DEA-2/Correlation.txt'
corr.to_csv(filepath,sep = '\t',header=False,index=False)
corr

1007 Number of observations
                                   mean    std   AMB0   AMB1    BSA DURATION  \
                    statistics                                                 
AMB0                              11.95  11.91   1.00   0.82   0.15    -0.25   
AMB1                               5.07   5.87   0.82   1.00   0.13    -0.19   
BSA                               -0.10   0.22   0.15   0.13   1.00    -0.17   
DURATION                           2.78   4.36  -0.25  -0.19  -0.17     1.00   
ATTENTION                          0.46   0.10   0.05   0.08   0.00    -0.05   
ASA                                0.10   0.33   0.05   0.05   0.13    -0.14   
Absorbed_slack                     1.97  37.88  -0.04  -0.04   0.01     0.01   
Unabsorbed_slack                   1.39   1.99  -0.20  -0.16  -0.22     0.09   
Absorptive_capacity                0.23   0.72  -0.08  -0.06  -0.42     0.05   
Market_commonality                 0.41   0.14   0.03   0.01  -0.09     0.06   
Size        

mean    std     AMB0     AMB1      BSA  \
                    statistics                                              
AMB0                              11.95  11.91     1.00     0.82     0.15   
                    p-value         NaN    NaN  (1.000)  (0.000)  (0.000)   
AMB1                               5.07   5.87     0.82     1.00     0.13   
                    p-value         NaN    NaN  (0.000)  (1.000)  (0.000)   
BSA                               -0.10   0.22     0.15     0.13     1.00   
                    p-value         NaN    NaN  (0.000)  (0.000)  (1.000)   
DURATION                           2.78   4.36    -0.25    -0.19    -0.17   
                    p-value         NaN    NaN  (0.000)  (0.000)  (0.000)   
ATTENTION                          0.46   0.10     0.05     0.08     0.00   
                    p-value         NaN    NaN  (0.309)  (0.067)  (0.988)   
ASA                                0.10   0.33     0.05     0.05     0.13   
                    p-value         NaN    NaN  (0.199)  (0.148)  (0.000)   
Absorbed_slack                     1.97  37.88    -0.04    -0.04     0.01   
                    p-value         NaN    NaN  (0.289)  (0.361)  (0.797)   
Unabsorbed_slack                   1.39   1.99    -0.20    -0.16    -0.22   
                    p-value         NaN    NaN  (0.000)  (0.000)  (0.000)   
Absorptive_capacity                0.23   0.72    -0.08    -0.06    -0.42   
                    p-value         NaN    NaN  (0.024)  (0.079)  (0.000)   
Market_commonality                 0.41   0.14     0.03     0.01    -0.09   
                    p-value         NaN    NaN  (0.459)  (0.769)  (0.009)   
Size                               9.37   1.76     0.41     0.36     0.34   
                    p-value         NaN    NaN  (0.000)  (0.000)  (0.000)   
Year                            2003.51   5.19    -0.19    -0.06     0.03   
                    p-value         NaN    NaN  (0.000)  (0.120)  (0.339)   

                               DURATION ATTENTION      ASA Absorbed_slack  \
                    statistics                                              
AMB0                              -0.25      0.05     0.05          -0.04   
                    p-value     (0.000)   (0.309)  (0.199)        (0.289)   
AMB1                              -0.19      0.08     0.05          -0.04   
                    p-value     (0.000)   (0.067)  (0.148)        (0.361)   
BSA                               -0.17      0.00     0.13           0.01   
                    p-value     (0.000)   (0.988)  (0.000)        (0.797)   
DURATION                           1.00     -0.05    -0.14           0.01   
                    p-value     (1.000)   (0.214)  (0.000)        (0.862)   
ATTENTION                         -0.05      1.00    -0.06           0.19   
                    p-value     (0.214)   (1.000)  (0.171)        (0.000)   
ASA                               -0.14     -0.06     1.00           0.04   
                    p-value     (0.000)   (0.171)  (1.000)        (0.277)   
Absorbed_slack                     0.01      0.19     0.04           1.00   
                    p-value     (0.862)   (0.000)  (0.277)        (1.000)   
Unabsorbed_slack                   0.09      0.05     0.11           0.01   
                    p-value     (0.015)   (0.332)  (0.003)        (0.771)   
Absorptive_capacity                0.05      0.10    -0.01           0.00   
                    p-value     (0.118)   (0.025)  (0.803)        (0.952)   
Market_commonality                 0.06     -0.06    -0.02          -0.02   
                    p-value     (0.094)   (0.201)  (0.544)        (0.662)   
Size                              -0.17      0.00     0.01          -0.10   
                    p-value     (0.000)   (0.945)  (0.722)        (0.014)   
Year                               0.19     -0.00    -0.02           0.07   
                    p-value     (0.000)   (0.947)  (0.532)        (0.069)   

                             

## Regression

In [112]:
R = []
mydata = df[['Id','AMB0','AMB1','BSA','DURATION','ATTENTION','ASA','Absorbed_slack','Unabsorbed_slack','Absorptive_capacity','Market_commonality','Size','Year']]
mydata = mydata.dropna()
mydata = mydata-mydata.mean()
mydata = mydata.sort_values(by=['Id','Year'], axis=0, ascending=True)
for amb in ['AMB0','AMB1']:
  reg = sm.OLS.from_formula(amb+'~BSA+DURATION+ATTENTION+BSA*ATTENTION+DURATION*ATTENTION+ASA+Absorbed_slack+Unabsorbed_slack+Absorptive_capacity+Market_commonality+Size+C(Id)+C(Year)',data=mydata).fit(cov_type='hac-panel',cov_kwds={'maxlags':5,'groups':mydata['Id']})   
  R.append((amb,reg.params,reg.bse,reg.pvalues,reg.rsquared_adj))
  print(reg.summary())


                            OLS Regression Results                            
Dep. Variable:                   AMB0   R-squared:                       0.829
Model:                            OLS   Adj. R-squared:                  0.789
Method:                 Least Squares   F-statistic:                     401.4
Date:                Thu, 11 Mar 2021   Prob (F-statistic):           3.75e-44
Time:                        06:22:47   Log-Likelihood:                -1122.5
No. Observations:                 365   AIC:                             2385.
Df Residuals:                     295   BIC:                             2658.
Df Model:                          69                                         
Covariance Type:            hac-panel                                         
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

In [113]:
table = Table_Summary(R)
table = table.reindex(['Intercept','BSA','DURATION','ATTENTION','BSA:ATTENTION','DURATION:ATTENTION']+['Adjusted R2'],level=0)
table = table.reindex(['Coeff','CSE','p-val',''],level=1)
table.round(2)

Measures                        AMB0   AMB1
Variables          Statistics              
Intercept          Coeff      -14.99 -11.42
                   CSE          4.49   1.97
                   p-val        0.00   0.00
BSA                Coeff        2.97   2.43
                   CSE          1.90   1.56
                   p-val        0.12   0.12
DURATION           Coeff       -0.24  -0.09
                   CSE          0.13   0.09
                   p-val        0.07   0.33
ATTENTION          Coeff        2.57   8.33
                   CSE         12.68   7.21
                   p-val        0.84   0.25
BSA:ATTENTION      Coeff        2.59 -28.07
                   CSE         26.35  17.56
                   p-val        0.92   0.11
DURATION:ATTENTION Coeff        2.35   0.80
                   CSE          2.02   1.05
                   p-val        0.24   0.45
Adjusted R2                     0.79   0.59